In [ ]:
from pathlib import Path
import glob as gb
import re
import os
from copy import deepcopy
import warnings

In [ ]:
ROOT_DIR = Path('.').absolute().parent.parent
ROOT_DIR

In [ ]:
ROOT_DIR = Path('/Users/julien/Software/Others/OpenStudio')

In [ ]:
#cpp_files = gb.glob(str(ROOT_DIR / 'src' / '**' / '*.cpp'), recursive=True)
cpp_files = list(ROOT_DIR.glob('src/**/*.cpp'))

# Initialize vertices directly

In [ ]:
#cpp_files = [ '/Users/julien/Software/Others/OpenStudio/src/model/test/SubSurface_GTest.cpp',]

In [ ]:
RE_VERTICES = re.compile(r'^(\s*)(?:std::vector<Point3d>|Point3dVector) (\w+);')
RE_VERTICES_CLEAR = re.compile(r'^(\s*)(\w+).clear\(\);')
RE_PUSHBACK = re.compile(r'\w+.push_back\(Point3d\(\s*(-?\d+(?:\.\d+)?),\s*(-?\d+(?:\.\d+)?),\s*(-?\d+(?:\.\d+)?)\)\);')
RE_PUSHBACK = re.compile(r'\w+.push_back\(Point3d\(\s*([^,]+),\s*([^,]+),\s*([^,]+)\)\);')

In [ ]:
for cpp_file in cpp_files:
    with open(cpp_file, 'r') as f:
        content = f.read()
    lines = content.splitlines()
    done_lines = []
    new_lines = []
    anyDone = False
    for i, line in enumerate(lines):
        if i in done_lines:
            continue
        done_lines.append(i)
        if (m11 := RE_VERTICES.search(line)) or (m12 := RE_VERTICES_CLEAR.search(line)):
            if m11:
                spacing = m11.groups()[0]
            else:
                spacing = m12.groups()[0]
            print(i, line)
            j = i+1
            pts = []
            while (m2 := RE_PUSHBACK.search(lines[j])):
                anyDone = True
                pts.append(m2.groups())
                done_lines.append(j)
                j += 1
            if pts:
                if m11:
                    new_lines.append(line.replace(';', ' {'))
                else:
                    new_lines.append(line.replace('.clear();', ' = {'))
                for k, pt in enumerate(pts):
                    new_line = spacing + '  {{{}, {}, {}}}'.format(*pt)
                    # if k != (len(pts) - 1):
                    new_line += ','
                    new_lines.append(new_line)

                new_lines.append(spacing + '};')
            else:
                for k in range(i, j):
                    new_lines.append(lines[i])
        else:
            new_lines.append(line)
        
    if anyDone:
        with open(cpp_file, 'w') as f:
            f.write("\n".join(new_lines) + '\n')
            

# std::string by const ref

In [ ]:
p_method_name = re.compile(r'(\s*)bool\s+(.*?_Impl::)(set.*?)(\(\s*(std::string).*)')

all_dict = {}
changed_dict = {}


# Debug
n_tot = 0

for file in cpp_files:
    
    # Check number of method changed for consistency between cpp / hpp/ Impl
    # Stores the actual line of function names for checking
    cpp_impl_methods = []
    
    class_name = os.path.splitext(os.path.basename(file))[0]
    with open(file, 'r') as f:
        ori_lines = f.read().splitlines()

    lines = deepcopy(ori_lines)
    # Might as well strip the trailing (right) whitespace too
    lines = [x.rstrip() for x in lines]
    
    found_lines = []
    for i, line in enumerate(lines):
        # Check that the method name is setXXX
        m3 = p_method_name.match(lines[i])
        
        if m3:
            method_name = m3.groups()[2]
            # print(i, m3.groups())
            # Go until the end of the function
            lines[i] = "{}bool {}{}{}".format(m3.groups()[0],
                                              m3.groups()[1],
                                              m3.groups()[2],
                                              m3.groups()[3].replace('std::string', 'const std::string&'))
            
            found_lines.append(i)
            cpp_impl_methods.append(line)
            if not class_name in changed_dict.keys():
                changed_dict[class_name] = [method_name]
            else:
                changed_dict[class_name].append(method_name)

            
    if found_lines:
        
        # Make the change to the Public implemention in the cpp file
        cpp_public_methods = []
        p_getimpl = re.compile(r'(\s*)(getImpl.*)')

        for method_name in changed_dict[class_name]:
            p_public = re.compile(r'(\s*)bool\s*({}::{}\s*\(\s*(std::string).*)'.format(class_name, method_name))
            for i, line in enumerate(lines):
                m = p_public.match(line)
                if m:
                    found_lines.append(i)
                    lines[i] = lines[i].replace('std::string', 'const std::string&')
                    cpp_public_methods.append(line)

                        
        all_dict[file] = {'ori': ori_lines, 'new': lines}
        # print('{}: {}'.format(file, found_lines))
        # if len(found_lines)%2 != 0:
        #     print("Something probably went wrong")
            
        # Write the cpp file
        with open(file, 'w') as f:
            f.write("\n".join(lines) + "\n")
            
        # Change it in the hpp files
        
        file_hpp = os.path.join(base_dir, '{}.hpp'.format(class_name))
        file_impl_hpp = os.path.join(base_dir, '{}_Impl.hpp'.format(class_name))
        
        hpp_impl_methods = []
        hpp_public_methods = []
        
        for file_header in [file_hpp, file_impl_hpp]:
            write_needed = False
            with open(file_header, 'r') as f:
                lines = f.read().splitlines()
            for method_name in changed_dict[class_name]:
                p = re.compile('(\s*)bool\s*({}\s*\(\s*(std::string).*)'.format(method_name))

                # Might as well strip the trailing (right) whitespace too
                lines = [x.rstrip() for x in lines]
                is_found = False
                for i, line in enumerate(lines):
                    m = p.match(line)
                    if m:
                        is_found = True
                        write_needed = True
                        lines[i] = lines[i].replace('std::string', 'const std::string&')
                        if file_header == file_hpp:
                            hpp_public_methods.append(line)
                        else:
                            hpp_impl_methods.append(line)
                        n_tot += 1
                if not is_found:
                    print('{}: {} not found'.format(file, method_name))
            
                    
            if write_needed:
                with open(file_header, 'w') as f:
                    f.write("\n".join(lines) + "\n")
                    
                    
        if len(cpp_impl_methods) != len(hpp_impl_methods):
            warnings.warn("Not maching for {} in Impl: Cpp={}, Hpp="
                          "{}".format(file, cpp_impl_methods,
                                      hpp_impl_methods))
        if len(cpp_public_methods) != len(hpp_impl_methods):
            warnings.warn("Not maching for {} in Public: Cpp={}, Hpp="
                          "{}".format(file, cpp_public_methods,
                                      hpp_impl_methods))

# Destructors

In [ ]:
impl_hpp_files = list(ROOT_DIR.glob('src/**/*_Impl.hpp'))
hpp_files = list(ROOT_DIR.glob('src/**/*.hpp'))
print(len(impl_hpp_files), len(hpp_files))
hpp_files = list(set(hpp_files) - set(impl_hpp_files))
print(len(impl_hpp_files), len(hpp_files))

In [ ]:
re_destructors = re.compile(r'~(\w+)\(\)')
re_destructor_cpp_non_trivial = re.compile(r'(\w+)::~\1()\(\)\s*{\s*(\w+.*)\s*}')
re_destructor_cpp_trivial = re.compile(r'(\w+)::~\1()\(\)\s*{\s*}')

## Find non trivial destructors

In [ ]:
for cpp_file in cpp_files:
    class_name = cpp_file.stem
    with open(cpp_file, 'r') as f:
        content = f.read()
        
    for x in re_destructor_cpp_non_trivial.finditer(content):
        print(f'{cpp_file}: {x.groups()}')

## Find trivial destructors defined in cpp file and move them to = default in hpp

In [ ]:
re_destructor_cpp_trivial = re.compile(r'^[\t ]*(\w+)::~\1()\(\)\s*{\s*}\s*?$[\r\n]', re.MULTILINE)

In [ ]:
for cpp_file in cpp_files:
#for cpp_file in [Path('/Users/julien/Software/Others/OpenStudio/src/airflow/contam/PrjAirflowElements.cpp')]:
    class_name = cpp_file.stem
    with open(cpp_file, 'r') as f:
        content = f.read()
    found_dtors = []
    for x in re_destructor_cpp_trivial.finditer(content):
        found_dtors.append(x.groups()[0])
    if found_dtors:
        #print(cpp_file, found_dtors)
        with open(cpp_file, 'w') as f:
            f.write(re_destructor_cpp_trivial.sub('', content))
    
        hpp_file = cpp_file.with_suffix('.hpp')
        impl_file = cpp_file.with_stem(cpp_file.stem + "_Impl").with_suffix('.hpp')
        impl_dtors = [x for x in found_dtors if '_Impl' in x]
        dtors = list(set(found_dtors) - set(impl_dtors))
        if impl_dtors:
            with open(impl_file, 'r') as f:
                content = f.read()
            lines = content.splitlines()
            new_lines = []
            for i, line in enumerate(lines):
                found = False
                for dtor in impl_dtors:
                    if f'~{dtor}();' in line:
                        found = True
                        new_lines.append(line.replace(';', ' = default;'))
                if not found:
                    new_lines.append(line)      
            with open(impl_file, 'w') as f:
                f.write('\n'.join(new_lines) + '\n')
                
        if dtors:       
            with open(hpp_file, 'r') as f:
                content = f.read()
            lines = content.splitlines()
            new_lines = []
            for i, line in enumerate(lines):
                found = False
                for dtor in dtors:
                    if f'~{dtor}();' in line:
                        found = True
                        new_lines.append(line.replace(';', ' = default;'))
                if not found:
                    new_lines.append(line)
            with open(hpp_file, 'w') as f:
                f.write('\n'.join(new_lines) + '\n')

## Replace all trivial dtors in hpp/Impl.hpp files by = default

In [ ]:
hpp_files = list(ROOT_DIR.glob('src/**/*.hpp'))
re_destructor_hpp_trivial = re.compile(r'^([\t ]*(?:virtual *)?~\w+\(\))(\s*?{\s*?}\s*?;*)(.*?)$', re.MULTILINE)

for hpp_file in hpp_files:
    with open(hpp_file, 'r') as f:
        content = f.read()
        


    found_dtors = []
    for x in re_destructor_hpp_trivial.finditer(content):
        #print(x.groups())
        found_dtors.append(x.groups()[1])
    if found_dtors:
        #print(hpp_file, found_dtors)
        with open(hpp_file, 'w') as f:
            f.write(re_destructor_hpp_trivial.sub(r'\1 = default;\3', content))

# Move in clone

In [88]:
cpp_model_files = list(ROOT_DIR.glob('src/model/*.cpp'))

re_normal_return = re.compile(r'\s*return\s*(\w+);(.*)')

re_init_return = re.compile(r'(\w+)\s+' + class_name + r'\s+=\s+(\w+)::clone\(model\)(?:.cast<(\w+)>\(\);)')

In [100]:
for cpp_file in cpp_model_files:
#for cpp_file in [Path('/Users/julien/Software/Others/OpenStudio/src/model/FanOnOff.cpp')]:
    
    class_name = cpp_file.stem
    with open(cpp_file, 'r') as f:
        content = f.read()
    if not f'{class_name}_Impl::clone(Model model)' in content:
        #print(f'{class_name} has no clone override')
        continue
        
    lines = content.splitlines()
    
    i = 0
    while (f'{class_name}_Impl::clone(Model model)' not in lines[i]):
        i += 1
        
    k = i + 1
    while ('return' not in lines[k]):
        k += 1
    
    if (m := re_normal_return.search(lines[k])):
        return_var, trailing_chars = m.groups()
        for j in range(i, k):
            re_init_return = re.compile(r'^(\s*)(\w+)\s+(' + return_var + r')\s+=\s+(\w+::clone\(model\)(?:.cast<(\w+)>\(\);.*)$')
            if (m := re_init_return.match(lines[j])):
                print(m.groups())
            if f'{return_var} =' in lines[j]:
                print(lines[j])
    #print('\n'.join(lines[i:k+1]) + '\n')

    continue
    in_block = False
    for i, line in enumerate(lines):
        if f'{class_name}_Impl::clone(Model model)' in line:
            in_block = True
        elif in_block and 'return' in line:
            print(line)
            print("\n")
            in_block = False
            continue
        
        if in_block:
            print(line)
            

('      ', 'auto', 't_clone', 'ParentObject_Impl::clone(model)', 'CoilHeatingWaterToAirHeatPumpVariableSpeedEquationFitSpeedData')
      auto t_clone = ParentObject_Impl::clone(model).cast<CoilHeatingWaterToAirHeatPumpVariableSpeedEquationFitSpeedData>();
('      ', 'GeneratorFuelCell', 'newFC', 'ModelObject_Impl::clone(model)', 'GeneratorFuelCell')
      GeneratorFuelCell newFC = ModelObject_Impl::clone(model).cast<GeneratorFuelCell>();
('      ', 'SolarCollectorFlatPlatePhotovoltaicThermal', 'result', 'StraightComponent_Impl::clone(model)', 'SolarCollectorFlatPlatePhotovoltaicThermal')
      SolarCollectorFlatPlatePhotovoltaicThermal result = StraightComponent_Impl::clone(model).cast<SolarCollectorFlatPlatePhotovoltaicThermal>();
('      ', 'auto', 't_clone', 'StraightComponent_Impl::clone(model)', 'CoilCoolingDXVariableSpeed')
      auto t_clone = StraightComponent_Impl::clone(model).cast<CoilCoolingDXVariableSpeed>();
('      ', 'auto', 'newCoil', 'StraightComponent_Impl::clone(mod